# **🏠 부동산 실거래가 예측**


# 1. Library Import

In [ ]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm 

font_path="G:/내 드라이브/NanumGothic.ttf" # 폰트의 원하는 이름 설정
fm.fontManager.addfont(font_path)           # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) # 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
from datetime import datetime

# Model
#from sklearnex import patch_sklearn
#patch_sklearn()
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import plot_importance
from lightgbm.sklearn import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn import metrics
import lightgbm as lgb

import eli5
from eli5.sklearn import PermutationImportance

# 추가한 라이브러리---------------------------------------------------------------------------------------------
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from math import radians, sin, cos, sqrt, atan2

import missingno as msno  # pip install missingno

import folium # pip install folium

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

import optuna
from optuna.samplers import TPESampler
from functools import partial

import joblib
import json


# 2. Data Load

In [ ]:
data_path='../data'
model_path = '../model'
submission_path = '../submission'

dt_train = pd.read_csv(f"{data_path}/train.csv")
dt_test = pd.read_csv(f"{data_path}/test.csv")

# 외부데이터
subway=pd.read_csv(f'{data_path}/origin/subway_origin.csv')
bus=pd.read_csv(f'{data_path}/origin/bus_origin.csv')
school=pd.read_csv(f'{data_path}/origin/school_origin.csv',encoding='cp949')
money = pd.read_csv(f'{data_path}/외부금리.csv')
apart_info = pd.read_parquet(f'{data_path}/아파트정보.parquet')
real_price=pd.read_csv(f'{data_path}/실거래가격지수.csv')
income = pd.read_csv(f'{data_path}/가구총소득.csv')

In [ ]:
print('Train data shape : ', dt_train.shape, '\nTest data shape : ', dt_test.shape)
display(dt_train.head(1))
display(dt_test.head(1))      

### 2-1. train,test 병합

In [ ]:
dt_train['is_test'] = 0
dt_test['is_test'] = 1

dt_full = pd.concat([dt_train, dt_test])   
dt_full.columns = [col.replace('(', '').replace(')', '').replace('㎡','').replace('~','_').replace('-','_').replace(',','').replace('/','').replace('=','_').replace(' ','_') for col in dt_full.columns]
dt_full.info()

# EDA

In [ ]:
# 전체 데이터 결측치 확인
msno.matrix(dt_full,figsize=(18,7), labels=True,fontsize=12)

# 중복 제거 결측치 확인
#msno.matrix(dt_full.drop_duplicates(subset=['시군구','아파트명']),figsize=(18,7), labels=True,fontsize=12)

In [ ]:
# 특정 컬럼을 기준으로 그룹화하여 실거래가 평균 보여주는 함수 
def group_viz(data,col,plot_type):
    """
    data=input_data
    col=시각화하고자 하는 컬럼
    plot_type= bar / plot 
    """
    tmp = data.groupby(col)['target'].mean().reset_index()
    fig=plt.figure(figsize=(10, 6))
    
    if(plot_type=='bar'):plt.bar(tmp[col], tmp['target'])
    else:plt.plot(tmp[col], tmp['target'], marker='o', linestyle='-', color='b')

    plt.xlabel(f'{col}별')
    plt.ylabel('평균')
    plt.title(f'{col}별 평균 실거래가')

    plt.xticks(rotation=45)
    plt.show()

### 3-1. 필요없는 컬럼 정리

In [ ]:
# columns='계약년월'
# group_viz(dt_full,columns,'bar')
# dt_full[columns].unique()
# dt_full[columns].info()

In [ ]:
drop_cols=["해제사유발생일","등기신청일자",'중개사소재지','k_단지분류아파트주상복합등등','k_전화번호', 'k_팩스번호', '단지소개기존clob','k_사용검사일_사용승인일',
           'k_홈페이지', 'k_등록일자','k_수정일자', '고용보험관리번호', '경비비관리형태','청소비관리형태', '기타의무임대임의_1234', '단지승인일', '사용허가여부','단지신청일']
dt_full.drop(drop_cols,axis=1,inplace=True)

In [ ]:
# 1차 컬럼 제거 후 남은 컬럼
dt_full.columns

## 3-2.경위도 결측값 처리

In [ ]:
# 기존 데이터에는 (좌표x,좌표y)를 보면 대부분 결측치 인것을 확인할 수 있음.
dt_full[['좌표X','좌표Y']].info()

In [ ]:
loc=dt_full[['시군구','번지', '아파트명','도로명', '좌표X', '좌표Y','target']]
loc_before= loc.drop_duplicates(subset=['시군구','아파트명'],keep='first')
loc_before.head(3)

In [ ]:
loc_before['임시주소']=loc_before['시군구']+" "+loc_before['번지']
loc_before.head(3)
#loc_before['임시주소'].to_csv("../data/loc_before.csv",index=False)
loc_after=pd.read_csv("../data/loc_after.csv")
loc_after[['시군구', '번지']] = loc_after['도로명'].str.rsplit(n=1, expand=True)
loc_after.head(3)

In [ ]:
loc_before=loc_before.dropna(subset='임시주소')
loc_before.reset_index(inplace=True,drop=True)
loc_after = pd.merge(loc_before, loc_after, left_index=True, right_index=True)
loc_after.head(3)
loc_after[['위도','경도']].info()

In [ ]:
# 지오 코딩을 해도 경위도가 결측치인 데이터들은 아래 2가지 방법을 이용하여 채움
# 1. 같은 시군구 아파트들의 경위도의 평균
# 2. 같은 아파트명 아파트들의 평균
missing_data = loc_after[loc_after['위도'].isnull()]
avg_sgg = loc_after.groupby('시군구_x').agg({'위도': 'mean', '경도': 'mean'}).reset_index()
for index, row in missing_data.iterrows():
    sgg = row['시군구_x']
    avg_lat = avg_sgg.loc[avg_sgg['시군구_x'] == sgg, '위도'].values[0]
    avg_lon = avg_sgg.loc[avg_sgg['시군구_x'] == sgg, '경도'].values[0]
    loc_after.loc[index, '위도'] = avg_lat
    loc_after.loc[index, '경도'] = avg_lon
    
missing_data = loc_after[loc_after['위도'].isnull()]
avg_apt = loc_after.groupby('아파트명').agg({'위도': 'mean', '경도': 'mean'}).reset_index()
for index, row in missing_data.iterrows():
    apt = row['아파트명']
    avg_lat = avg_apt.loc[avg_apt['아파트명'] == apt, '위도'].values[0]
    avg_lon = avg_apt.loc[avg_apt['아파트명'] == apt, '경도'].values[0]
    loc_after.loc[index, '위도'] = avg_lat
    loc_after.loc[index, '경도'] = avg_lon

In [ ]:
# 기존 경위도가 결측치 인것들에 한해서만 지오코딩의 결과값으로 바꿈
# 지오코딩의 아주 약간의 오차가 있기 때문.(-> 오픈스트리트맵으로 확인 결과 실제 오차는 없음)
loc_after['좌표X'] = loc_after['좌표X'].fillna(loc_after['경도'])
loc_after['좌표Y'] = loc_after['좌표Y'].fillna(loc_after['위도'])
loc_after.drop(['임시주소','도로명_y','위도','경도','시군구_y','번지_y','target','도로명_x'],axis=1,inplace=True)
loc_after.columns=['시군구', '번지', '아파트명', '좌표X', '좌표Y']

In [ ]:
# 경위도 결측치가 채워진 데이터
#loc_after.to_csv("../data/filled_loc.csv",index=False)
#loc_after=pd.read_csv('../data/filled_loc.csv')

## 3-3. Feature engineering

In [ ]:
# 본번, 부번 범주형으로 변환

dt_full['본번'] = dt_full['본번'].astype('str')
dt_full['부번'] = dt_full['부번'].astype('str')

### 3-3-1. 계약년도, 월, 일

In [ ]:
# 계약년도를 년,월로 분할
dt_full['계약년도'] = dt_full['계약년월'].astype(str).str[:4].astype(int)
dt_full['계약월'] = dt_full['계약년월'].astype(str).str[4:].astype(int)
dt_full[['계약년월','계약년도','계약월']].head(3)

In [ ]:
# 계약년월별 실거래가 시각화
df=dt_full.copy()
df['날짜'] = pd.to_datetime(df['계약년도'].astype(str) + '-' + df['계약월'].astype(str))
df = df[['날짜', 'target']]
grouped_data = df.groupby('날짜')['target'].mean().reset_index()
plt.figure(figsize=(15, 6))
sns.lineplot(x='날짜', y='target', data=grouped_data)
plt.xlabel('날짜')
plt.ylabel('실거래가격 평균')
plt.title('평균 실거래가')
plt.show()
del df;

### 3-3-2. 외부 데이터 결합(경위도 기준)

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (단위: km)

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)

    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

def near_facility(data,u_dist,type,n,target_col):
    """
    함수를 사용하기 전 참조외부데이터에 위도,경도 column이 존재해야함.
    인접거리를 정해주고, 개수를 반환할지 이름을 반환할지 결정하여 u_dist와 type 설정
    target_col도 data에 있는 컬럼이어야 함.
    
    
    data=참조 데이터(버스,지하철,학교 등)
    u_dist=인접거리(소수로.)
    type=count:개수반환/name:이름반환
    n=인접개수
    target_col=참조 컬럼
    """
    origin_data=loc_after.copy()
    for i, origin_coord in enumerate(origin_data[['좌표Y', '좌표X']].values):
        distances = [haversine(origin_coord[0], origin_coord[1], tmp_coord[0], tmp_coord[1]) for tmp_coord in data[['위도', '경도']].values]
        # u_dist 거리 내에 있는 시설을 최대 n개 찾기
        
        if(type=='name'):
            closest_facility= [idx for idx, dist in sorted(enumerate(distances), key=lambda x: x[1]) if dist <= u_dist][:n]
            # 계산된 인접 시설을 추가 
            for j, idx in enumerate(closest_facility):
                for k in target_col:
                    col_name = f'인접{k}{j+1}'
                    origin_data.at[i, col_name] = data.iloc[idx][k]
        else:
            closest_facility= [idx for idx, dist in sorted(enumerate(distances), key=lambda x: x[1]) if dist <= u_dist]
            col_name = f'인접{target_col[0]}개수'
            origin_data.at[i, col_name] = len(closest_facility)
                
    return origin_data

#### 3-3-2-1.지하철

In [ ]:
subway.info(),subway.head(3)

In [ ]:
# tmp=near_facility(data=subway,u_dist=0.7,type='name',n=3,target_col=['역사명','호선'])
# tmp.head(3)
# loc_after[['인접역사명1', '인접호선1', '인접역사명2', '인접호선2','인접역사명3', '인접호선3']]=tmp[['인접역사명1', '인접호선1', '인접역사명2', '인접호선2','인접역사명3', '인접호선3']]
# loc_after.head(3)

tmp=near_facility(data=subway,u_dist=0.8,type='len',n=None,target_col=['역사명']) # 800m인접, 개수 반환
tmp['인접역사명개수']=tmp['인접역사명개수'].fillna(0)
loc_after[['인접역사명개수']]=tmp[['인접역사명개수']].astype(int)
loc_after.head(3)

#### 3-3-2-2.학교

In [ ]:
# https://data.seoul.go.kr/dataList/OA-20502/S/1/datasetView.do
school=pd.read_csv("../data/origin/school_origin.csv",encoding='cp949')
school=school[['학교종류명','설립구분','학교명','도로명주소','도로명상세주소']]
school.head(3)
#school.to_csv("../data/school_before.csv",index=False)
# Naver 지오코딩 api를 이용해 주소->경위도 변환 
school_after=pd.read_csv("../data/school_after.csv") # 후처리된 school
school[['위도','경도']]=school_after[['위도','경도']]
school.head(3)

In [ ]:
# tmp=near_facility(data=school,u_dist=0.9,type='name',n=2,target_col=['학교명','학교종류명'])
# loc_after[['인접학교명1', '인접학교종류명1', '인접학교명2', '인접학교종류명2']]=tmp[['인접학교명1', '인접학교종류명1', '인접학교명2', '인접학교종류명2']]
# loc_after.head(3)

tmp=near_facility(data=school,u_dist=0.8,type='len',n=None,target_col=['학교명'])
tmp['인접학교명개수']=tmp['인접학교명개수'].fillna(0)
loc_after[['인접학교명개수']]=tmp[['인접학교명개수']].astype(int)
loc_after.head(3)
# 잘 붙여졌는지 확인하려면 아래 링크에 해당 아파트를 검색하여 인근에 붙여진 학교를 확인
# https://hogangnono.com/apt/1OKb7/0

#### 3-3-2-3.버스

In [ ]:
bus.info(),bus.head(3)

In [ ]:
bus.columns=['노드 ID', '정류소번호', '정류소명', '경도', '위도', '정류소 타입']

In [ ]:
tmp=near_facility(data=bus,u_dist=0.3,type='len',n=None,target_col=['정류소명'])
tmp['인접정류소명개수']=tmp['인접정류소명개수'].fillna(0)
loc_after[['인접정류소명개수']]=tmp[['인접정류소명개수']].astype(int)
loc_after.head(3)

In [ ]:
dt_full.drop(['좌표X','좌표Y'],axis=1,inplace=True)
dt_full= pd.merge(dt_full, loc_after, on=['시군구', '번지','아파트명'], how='left')
dt_full.columns,dt_full.head(3)

In [ ]:
dt_full_bak=dt_full.copy()

#### 3-3-2-4. 금리

In [ ]:
money.drop(columns=['통계표', '단위', '변환'], inplace=True)

money.columns = [''.join(x) for x in money.columns.str.split('/')]

money = pd.melt(money, id_vars=['계정항목'], var_name='년월', value_name='금리')
money['년월'] = money['년월'].astype(np.int32)

money = money.pivot(index='년월', columns='계정항목', values='금리').reset_index()

money = money[['년월', '정부대출금금리', '한국은행 기준금리'] ]
money.columns = ['계약년월', '대출금리', '기준금리']

money.loc[money['계약년월'] > 202306,'대출금리'] = money[money['계약년월'] == 202306]['대출금리'].iloc[0]
money.loc[money['계약년월'] > 202306,'기준금리'] = money[money['계약년월'] == 202306]['기준금리'].iloc[0]

dt_full = dt_full.merge(money, how='left', on='계약년월')

#### 3-3-2-5. 실거래지수

In [ ]:
real_price['년월']=real_price['년월'].astype('str').apply(lambda x: '{:.2f}'.format(float(x)))
real_price['년월'] = real_price['년월'].apply(lambda x: datetime.strptime(x, "%Y.%m"))
real_price['계약월']=real_price['년월'].dt.month
real_price['계약년도']=real_price['년월'].dt.year
real_price.head(10)

In [ ]:
plt.figure(figsize=(20, 6)) 
plt.plot(real_price['년월'], real_price['실거래지수'], marker='o', linestyle='-') 
plt.title('실거래지수 변화')  
plt.xlabel('년월') 
plt.ylabel('실거래지수') 
plt.grid(True) 
plt.xticks(rotation=45)  
plt.show();

In [ ]:
# 2023년 7월 이후의 실거래지수는 사용하면 안되기 때문에 6월의 실거래지수를 그대로 반영 
real_price.loc[(real_price['계약년도'] == 2023) & (real_price['계약월'] >= 7), '실거래지수']=real_price[real_price['년월']=='2023-06-01']['실거래지수'].values[0]
real_price[(real_price['계약년도']==2023) & (real_price['계약월']>=7)]

dt_full['계약월']=dt_full['계약월'].astype('int')
dt_full=pd.merge(dt_full,real_price[['계약년도','계약월','실거래지수']],on=['계약년도','계약월'],how='left')
dt_full[['계약년도','계약월','실거래지수']]

#### 3-3-2-6. 아파트정보 결측치 처리

In [ ]:
apart_info = pd.read_parquet(f"{data_path}/아파트정보.parquet")

In [ ]:
apart_info = apart_info[apart_info['시도'] == '서울특별시']
apart_info['법정주소'] = apart_info['법정동주소'].str.split(",").str[0]
apart_info['아파트주소'] = apart_info['법정주소'].str.split(" ").str[0:4].apply(lambda x: " ".join(x))
apart_info['승강기'] = apart_info[['승강기(승객용)', '승강기(화물용)', '승강기(승객+화물)','승강기(장애인)', '승강기(비상용)', '승강기(기타)']].max(axis=1)

apart_info['차량보유대수(전체)'] = apart_info['차량보유대수(전체)'].astype('Int64')
apart_info['차량보유대수(전기차)'] = apart_info['차량보유대수(전기차)'].astype('Int64')

apart_info['전기차전용주차면수(지상)'] = apart_info['전기차전용주차면수(지상)'].replace("해당없음", "0")
apart_info['전기차전용주차면수(지하)'] = apart_info['전기차전용주차면수(지하)'].replace("해당없음", "0")

apart_info['전기차전용주차면수(지상)'] = apart_info['전기차전용주차면수(지상)'].astype("Int64")
apart_info['전기차전용주차면수(지하)'] = apart_info['전기차전용주차면수(지하)'].astype(float)

apart_info['전기충전기설치여부(지상)'] = apart_info['전기충전기설치여부(지상)'].map({"해당없음":0, "설치":1}).astype("Int64")
apart_info['전기충전기설치여부(지하)'] = apart_info['전기충전기설치여부(지하)'].map({"해당없음":0, "설치":1}).astype("Int64")
apart_info['전기충전기설치여부'] = apart_info['전기충전기설치여부(지상)'] | apart_info['전기충전기설치여부(지하)']

# 전기차전용주차면수(지하) 컬럼 이상치 처리
apart_info.loc[apart_info['단지코드'] == "A10023343", '전기차전용주차면수(지하)'] = apart_info.loc[apart_info['단지코드'] == "A10023343", '총주차대수']

apart_info['전기차전용주차면수(지하)'] = apart_info['전기차전용주차면수(지하)'].astype('Int64')

apart_info['전기차전용주차면수'] = apart_info['전기차전용주차면수(지상)'] + apart_info['전기차전용주차면수(지하)']

In [ ]:
basic_du = apart_info[apart_info.duplicated(subset='아파트주소', keep=False)]
basic_du['이전분양형태'] = basic_du.groupby('아파트주소')['분양형태'].shift(-1)
basic_du = basic_du[~((basic_du['분양형태'].str.contains("분양|혼합")) & basic_du['이전분양형태'].str.contains("분양|혼합"))]

apart_info.drop_duplicates(subset='아파트주소', keep=False, inplace=True)

In [ ]:
selected = ['분양형태', '법정주소', '단지명', '동수', '세대수', '분양세대수', '임대세대수', '관리방식', '난방방식', '복도유형', '시공사', '시행사',
'일반관리-관리방식', '일반관리-인원', '경비관리-관리방식', '경비관리-인원', '청소관리-관리방식', '청소관리-인원', '건물구조', '전기-수전용량', '전기-세대전기계약방식',
'승강기', '총주차대수', '지상주차대수', '지하주차대수', 'CCTV대수', '부대복리시설', '최고층수','지하층수', '차량보유대수(전체)',
'차량보유대수(전기차)', '전기충전기설치여부(지상)', '전기충전기설치여부(지하)','전기차전용주차면수(지상)','전기차전용주차면수(지하)', '아파트주소']

In [ ]:
AGGREGATIONS = {
    '분양형태': 'first',
    '법정주소': 'first',
    '단지명': 'first',
    '동수': 'sum',
    '세대수': 'sum',
    '분양세대수': 'sum',
    '임대세대수': 'sum',
    '관리방식': 'first',
    '난방방식': 'first',
    '복도유형': 'first',
    '시공사': 'first',
    '시행사': 'first',
    '일반관리-인원': 'sum',
    '청소관리-인원': 'sum',
    '전기-수전용량': 'sum',
    '전기-세대전기계약방식': 'first',
    '승강기': 'sum',
    '총주차대수': 'sum',
    '지상주차대수': 'sum',
    '지하주차대수': 'sum',
    'CCTV대수': 'sum',
    '부대복리시설': 'first',
    '최고층수': 'max',
    '지하층수': 'max',
    '차량보유대수(전체)': 'sum',
    '차량보유대수(전기차)': 'sum',
    '전기충전기설치여부(지상)': 'max',
    '전기충전기설치여부(지상)': 'max',
    '전기차전용주차면수(지상)': 'sum',
    '전기차전용주차면수(지하)': 'sum',
}

basic_du = basic_du[selected].groupby('아파트주소').agg(AGGREGATIONS).reset_index()
basic_du['분양형태'] = '혼합'

apart_info = pd.concat([apart_info, basic_du], axis=0)

In [ ]:
dt_full['아파트주소'] = dt_full['시군구'] + " " + dt_full['번지']
dt_full = dt_full.merge(apart_info[selected], how='left', on='아파트주소')
dt_full.shape

In [ ]:
dt_full.columns

In [ ]:
# 카테고리가 달라서 맵핑해주기
dt_full['분양형태'] = dt_full['분양형태'].map({"분양": "분양", "혼합": "기타", "임대": "임대"})
index = dt_full[dt_full['k_세대타입분양형태'].isna()].index
dt_full.loc[index, 'k_세대타입분양형태'] = dt_full.loc[index, '분양형태']

# 카테고리가 달라서 맵핑해주기
dt_full['관리방식'] = dt_full['관리방식'].map({"위탁관리": "위탁관리", "자치관리": "자치관리", "위탁관리(직영+위탁)": "직영", "자치관리(직영)": "직영", "위탁관리(총액관리제)": "직영", "관리방식미정":np.nan})
index = dt_full[dt_full['k_관리방식'].isna()].index
dt_full.loc[index, 'k_관리방식'] = dt_full.loc[index, '관리방식']

index = dt_full[dt_full['k_복도유형'].isna()].index
dt_full.loc[index, 'k_복도유형'] = dt_full.loc[index, '복도유형']

index = dt_full[dt_full['k_난방방식'].isna()].index
dt_full.loc[index, 'k_난방방식'] = dt_full.loc[index, '난방방식']

index = dt_full[dt_full['k_전체동수'].isna()].index
dt_full.loc[index, 'k_전체동수'] = dt_full.loc[index, '동수']

index = dt_full[dt_full['k_전체세대수'].isna()].index
dt_full.loc[index, 'k_전체세대수'] = dt_full.loc[index, '세대수']

index = dt_full[dt_full['k_건설사시공사'].isna()].index
dt_full.loc[index, 'k_건설사시공사'] = dt_full.loc[index, '시공사']

index = dt_full[dt_full['k_시행사'].isna()].index
dt_full.loc[index, 'k_시행사'] = dt_full.loc[index, '시행사']

index = dt_full[dt_full['세대전기계약방법'].isna()].index
dt_full.loc[index, '세대전기계약방법'] = dt_full.loc[index, '전기-세대전기계약방식']

# 원본 데이터에 이상치가 너무 많아서 조금 제거
dt_full[dt_full['주차대수'] < dt_full['k_전체세대수'].quantile(0.01)]['주차대수'] = np.nan

index = dt_full[dt_full['주차대수'].isna()].index
dt_full.loc[index, '주차대수'] = dt_full.loc[index, '총주차대수']

In [ ]:
drop_cols2 = ['분양형태', '관리방식', '일반관리-관리방식', '경비관리-관리방식', '청소관리-관리방식', '건물구조',
             '복도유형', '난방방식', '동수', '세대수', '시공사', '시행사', '전기-세대전기계약방식', 
             '총주차대수', '부대복리시설', '전기충전기설치여부(지상)', '전기충전기설치여부(지하)', 
             '전기차전용주차면수(지상)', '전기차전용주차면수(지하)']
dt_full.drop(columns=drop_cols2, inplace=True)
dt_full.columns

In [ ]:
dt_full["계약년월일"] = dt_full["계약년월"].astype(str) + dt_full["계약일"].astype(str)
dt_full["계약년월일"] = pd.to_datetime(dt_full['계약년월일'], format='%Y%m%d')

dt_full['계약년'] = dt_full['계약년월'].astype('str').map(lambda x : x[:4])
dt_full['계약월'] = dt_full['계약년월'].astype('str').map(lambda x : x[4:])

dt_full['풀아파트명'] = dt_full['시군구'] + " " + dt_full['아파트명']

dt_full['주소'] = dt_full['시군구'] + " " + dt_full['번지']

dt_full['구'] = dt_full['시군구'].map(lambda x : x.split()[1])
dt_full['동'] = dt_full['시군구'].map(lambda x : x.split()[2])

In [ ]:
dt_full['평단가'] = dt_full['target'] / dt_full['전용면적'] * 0.3025

target_encoding = dt_full.groupby('구')['평단가'].mean()
dt_full['구평단가'] = dt_full['구'].map(target_encoding)

target_encoding = dt_full.groupby('동')['평단가'].mean()
dt_full['동평단가'] = dt_full['동'].map(target_encoding)

dt_full['구순위'] = dt_full['구평단가'].rank(method='dense')
dt_full['동순위'] = dt_full['동평단가'].rank(method='dense')

target_encoding = dt_full.groupby('풀아파트명')['평단가'].mean()
dt_full['아파트평단가'] = dt_full['풀아파트명'].map(target_encoding)
dt_full['아파트순위'] = dt_full['아파트평단가'].rank(method='dense')

In [ ]:
dt_full['계약년월3'] = pd.to_datetime(dt_full['계약년월일']).dt.to_period("Q").dt.to_timestamp()
temp = dt_full.groupby(['풀아파트명', '계약년월3'])['평단가'].mean().reset_index(name="3월평단가")

AGG = {'3월평단가': [ 'std', 'skew', ('kurt', lambda x: x.kurt())]}
temp = temp.groupby('풀아파트명').agg(AGG)
temp.columns = ['_'.join(col).strip() for col in temp.columns.values]
dt_full = dt_full.merge(temp, on='풀아파트명', how='left')
del dt_full['계약년월3']

In [ ]:
gangnam = ['강서구', '영등포구', '동작구', '서초구', '강남구', '송파구', '강동구']

# '구' 열에 대해 강남 여부를 나타내는 '강남여부' 열 생성
dt_full['강남여부'] = dt_full['구'].apply(lambda x: 1 if x in gangnam else 0)

In [ ]:
dt_full['신축여부'] = dt_full['건축년도'].apply(lambda x: 1 if x >= 2009 else 0)

In [ ]:
dt_full['세대당주차대수'] = dt_full['주차대수'] / dt_full['k_전체세대수']
dt_full['세대당승강기대수'] = dt_full['승강기'] / dt_full['k_전체세대수'] 
dt_full['고층도'] = dt_full['층'] / dt_full['최고층수']
dt_full['동세대수'] = dt_full['k_전체세대수'] / dt_full['k_전체동수']

In [ ]:
# 월별 건수
temp = dt_full.groupby(['풀아파트명', '계약년월']).size().reset_index(name='월거래건수')
dt_full = dt_full.merge(temp, how='left', on=['풀아파트명', '계약년월'])

AGG = {'월거래건수': ['mean', 'std', 'skew', ('kurt', lambda x: x.kurt())]}
temp = temp.groupby('풀아파트명').agg(AGG)
temp.columns = ['_'.join(col).strip() for col in temp.columns.values]

dt_full = dt_full.merge(temp, on='풀아파트명', how='left')

In [ ]:
# 최근 5년 건수
train_max_date = dt_full[dt_full["is_test"] == 0]['계약년월일'].max()
lag5year = dt_full[dt_full['계약년월일'] > train_max_date - pd.DateOffset(years=5)]

target_encoding = lag5year.groupby('풀아파트명')['계약년월일'].size()
dt_full['아파트거래횟수5'] = dt_full['풀아파트명'].map(target_encoding)

In [ ]:
AGG = {'전용면적':['mean', 'std', ('kurt', lambda x: x.kurt()), 'size']}
temp = dt_full.groupby('풀아파트명').agg(AGG)
temp.columns = ['_'.join(col).strip() for col in temp.columns.values]
dt_full = dt_full.merge(temp, how='left', on='풀아파트명')

In [ ]:
dt_full['전용면적범주'] = pd.cut(dt_full['전용면적'], bins=[0, 40, 60, 85, 135, float('inf')], labels=[0, 1, 2, 3, 4], right=False)
dt_full['전용면적범주']=dt_full['전용면적범주'].astype('int')

temp = dt_full.pivot_table(index='풀아파트명', columns='전용면적범주', aggfunc='size', fill_value=0)
temp.columns = [temp.columns.name+"_"+str(col) for col in temp.columns.values]
dt_full = dt_full.merge(temp, how='left', on='풀아파트명')

dt_full['범주4비율'] = dt_full['전용면적범주_4'] / dt_full['전용면적_size']

In [ ]:
temp = dt_full.copy()
temp = temp.sort_values(by=['풀아파트명', '계약년월일'])
temp['아파트이전계약일'] = temp.groupby('풀아파트명')['계약년월일'].shift(1)
dt_full.loc[temp.index, '아파트이전계약일'] = temp['아파트이전계약일']
dt_full['이전계약_diff'] =  (dt_full['계약년월일'] - dt_full['아파트이전계약일']).dt.days

del dt_full['아파트이전계약일']

#### 3-3-2-7. 권역별 평균 소득

In [ ]:
def map_region(district):

    region_mapping = {
    '동남권': ['강남구', '서초구', '송파구', '강동구'],
    '서남권': ['동작구', '관악구', '금천구', '영등포구', '구로구', '양천구', '강서구'],
    '서북권': ['은평구', '서대문구', '마포구'],
    '도심권': ['종로구', '중구', '용산구'],
    '동북권': ['성동구', '광진구', '동대문구', '중랑구','성북구', '강북구', '도봉구', '노원구']
    }

    for region, districts in region_mapping.items():
        if district in districts:
            return region
    return '기타'  # 만약 매핑되지 않는 구가 있다면 '기타'로 분류

In [ ]:
df_5_regions = income[income['특성별(1)'] == '5권역별']
dt_full['권역'] = dt_full['구'].map(map_region)

sorted_df = df_5_regions.sort_values(by='2020.7', ascending=False)
sorted_df['2020.7'] =sorted_df['2020.7'].astype('float')  


In [ ]:
income_data = sorted_df.set_index('특성별(2)')['2020.7'].to_dict()
min_income = min(income_data.values())
max_income = max(income_data.values())

def normalize_income(income):
    return (income - min_income) / (max_income - min_income)

normalized_scores = {region: normalize_income(income) for region, income in income_data.items()}

dt_full['income'] = dt_full['권역'].map(normalized_scores)
dt_full.drop(columns=['권역'], inplace=True)

### 3-3-3. 최근거래가 추가

In [ ]:
def process_group(group):
    matching_rows = group.sort_values(by=['계약년월일'])[['idx', '아파트명', '전용면적범주', 'target', '계약년월일']]
    matching_rows['최근거래가'] = matching_rows['target'].shift(1)
    return matching_rows

dt_full['idx']=dt_full.index
dt_tmp = []
for _, group in tqdm(dt_full.groupby(['구', '아파트명', '전용면적범주'])):
    dt_tmp.append(process_group(group))
dt_tmp = pd.concat(dt_tmp, ignore_index=True)
dt_tmp.head(3)

In [ ]:
dt_full = pd.merge(dt_full, dt_tmp[['idx', '최근거래가']], left_on='idx', right_on='idx', how='left')
dt_full.reset_index(drop=True, inplace=True)
#dt_full[dt_full['아파트명']=='반포자이'].sort_values(by='계약일자')[['전용면적','계약일자','target','최근거래가']].head(50)
dt_full

In [ ]:
dt_full['최근거래가'] = dt_full['최근거래가'].fillna(dt_full.groupby(['아파트명', '동','전용면적범주'])['target'].transform('mean'))
#dt_full2[dt_full2['아파트명']=='개포6차우성'].sort_values(by='계약일자')[['전용면적','계약일자','target','최근거래가']]
dt_full['최근거래가'] = dt_full['최근거래가'].fillna(dt_full.groupby(['아파트명', '동'])['target'].transform('mean'))
dt_full['최근거래가'] = dt_full['최근거래가'].fillna(dt_full.groupby(['동'])['target'].transform('mean'))
dt_full[dt_full['최근거래가'].isna()==1]
dt_full.drop(['idx'],axis=1,inplace=True)

In [ ]:
dt_full_bak2=dt_full.copy()

## 3-4. 결측치 채우기

In [ ]:
dt_full.isnull().sum()

In [ ]:
num_col = []
cat_col = []
rest = []

for column in dt_full.columns:
    if pd.api.types.is_numeric_dtype(dt_full[column]):
        num_col.append(column)
    elif pd.api.types.is_object_dtype(dt_full[column]):
        cat_col.append(column)
    else:
        rest.append(column)

print("연속형 변수:", num_col)
print("범주형 변수:", cat_col)
print("나머지 변수:", rest)

In [ ]:
dt_full = dt_full.replace('-', np.nan)

for col in cat_col:
    mode_fill_value = dt_full[col].mode().iloc[0]
    dt_full[col] = dt_full[col].fillna(mode_fill_value)

# mean_cols에 있는 각 컬럼들의 결측치를 해당 컬럼의 평균값으로 채우기
for col in num_col:
    try:
        mean_fill_value = dt_full[col].mean()
        dt_full[col] = dt_full[col].fillna(mean_fill_value)
    except:pass
dt_full.isnull().sum()

In [ ]:
drop_cols3 = ['계약년월',"계약년도",
             '평단가', '동평단가', '구평단가', '아파트평단가',
             '아파트주소', '법정주소', '단지명',
             '건축면적', 'k_세대타입분양형태', '월거래건수',
             ]
dt_full.drop(columns=drop_cols3, inplace=True)
dt_full.columns
dt_full_bak3=dt_full.copy()

In [ ]:
dt_full_bak3.to_csv(f"{data_path}/dt_full_모델링.csv",index=False)

# 4. Model Training

## 모델 저장 및 로드

In [ ]:
def save_model(name,model):
    with open(f'{model_path}/{name}.pkl', 'wb') as f:
        pickle.dump(model, f)
    print("Save Success")
    
def load_model(name):
    with open(f'{model_path}/{name}.pkl', 'rb') as f:
        model = pickle.load(f)
    return model

## 4-1. LightGBM

In [ ]:
dt_full=pd.read_csv(f"{data_path}/dt_full_모델링.csv")

Logtarget=True # 타겟과 최근거래가를 로그화할지 말지

### 4-1-1.타겟 로그화

In [ ]:
if(Logtarget==True):
    print("Target and recenttarget log transform")
    dt_full['target']=np.log1p(dt_full['target'])
    try: dt_full['최근거래가']=np.log1p(dt_full['최근거래가'])
    except:pass

def transform_log(pred,target=None):
    if(Logtarget==True):
        return np.expm1(pred),np.expm1(target)
    else:
        return pred,target

### 4-1-1.년도 제거

In [ ]:
# dt_full=dt_full[dt_full['계약년도']>2019]
# dt_full.reset_index(drop=True,inplace=True)

### 4-1-3.최종 drop 컬럼 결정하기 

In [ ]:
dt_full.columns

In [ ]:
drop_cols4=['최근거래가','실거래지수']
dt_full.drop(drop_cols4,axis=1,inplace=True)

In [ ]:
dt_full.columns

### 4-1-4. Train, Test 분리

In [ ]:
dt_train = dt_full[dt_full['is_test']==0]
dt_test = dt_full[dt_full['is_test']==1]
dt_test['target'] = 0

dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)
assert dt_train.shape[1] == dt_test.shape[1] 

### 4-1-5. 라벨 인코딩

In [ ]:
num_cols,cat_cols,rest = [],[],[]

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]): num_cols.append(column) # 
    elif pd.api.types.is_object_dtype(dt_train[column]): cat_cols.append(column)
    else: rest.append(column)
    
print("연속형 변수:", num_cols)
print("범주형 변수:", cat_cols)
print("나머지 변수:", rest)

label_encoders = {}

for col in tqdm(cat_cols):
    if(col=='계약년월일'):continue
    lbl = LabelEncoder()
    lbl.fit( dt_train[col].astype(str) )
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    label_encoders[col] = lbl          
    for label in np.unique(dt_test[col]):
        if label not in lbl.classes_: 
            lbl.classes_ = np.append(lbl.classes_, label) 
    dt_test[col] = lbl.transform(dt_test[col].astype(str))
;

### 4-1-6. LGBM 모델 학습 및 검증

In [ ]:
X_train = dt_train[dt_train['계약년월일'] < '2023-01-01'].drop(columns=['target', '계약년월일'])
X_val = dt_train[dt_train['계약년월일'] >= '2023-01-01'].drop(columns=['target', '계약년월일'])
y_train = dt_train[dt_train['계약년월일'] < '2023-01-01']['target']
y_val = dt_train[dt_train['계약년월일'] >= '2023-01-01']['target']

In [ ]:
gbm = lgb.LGBMRegressor(n_estimators=100000,                # early stopping을 적용하기에 적당히 많은 반복 횟수를 지정합니다.
                        metric="rmse",
                        data_sample_strategy='goss',        # sampling 방법을 goss로 적용합니다.
                        max_depth=15,                       # default값인 20에서 12로 변경합니다.
                        num_leaves=2**7-1,                  # default값인 31에서 62으로 변경합니다. 2**depth > 2**5-1
                        # min_data_in_leaf=40,              # default값인 20에서 40으로 변경합니다.
                        min_child_samples=40,               # default값인 20에서 40으로 변경합니다.
                        colsample_bytree= 0.7,
                        subsample=0.7,
                        learning_rate=0.15,
                        random_state=42,
                        )
gbm.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_metric ='rmse',
        categorical_feature="auto",
        callbacks=[lgb.early_stopping(stopping_rounds=50),         # early stopping을 적용합니다. 50번동안 metirc의 개선이 없다면 학습을 중단합니다.
                   lgb.log_evaluation(period=10, show_stdv=True)]  # 10번의 반복마다 평가점수를 로그에 나타냅니다.
)
model_path='../model' 
save_model("lgbm_log",gbm)

In [ ]:
val_pred=gbm.predict(X_val)
val_pred,y_val=transform_log(val_pred,y_val)
rmse = np.sqrt(mean_squared_error(y_val, val_pred))

plt.figure(figsize=(20, 6))
plt.title("검증 데이터")
plt.text(0.5, 0.9, f'RMSE: {rmse:.2f}', fontsize=20, ha='center', transform=plt.gca().transAxes)
sns.lineplot(y_val.reset_index(drop=True), label='target', alpha=1)
sns.lineplot(val_pred, label='pred', alpha=1)
plt.grid()
plt.show();

In [ ]:
importances = pd.Series(gbm.feature_importances_, index=list(X_train.columns))
importances = importances.sort_values(ascending=False)

plt.figure(figsize=(10, 16))
plt.title("Feature Importances")
sns.barplot(x=importances, y=importances.index)
plt.show();

### 4-1-7. LGBM 모델 추론

In [ ]:
X_test = dt_test.drop(columns=['target', '계약년월일'])
test_pred = gbm.predict(X_test)
test_pred,_=transform_log(test_pred,0)
rmse = np.sqrt(mean_squared_error(y_test, test_pred))

plt.figure(figsize=(20, 6))
plt.title("테스트 데이터")
plt.text(0.5, 0.9, f'RMSE: {rmse:.2f}', fontsize=20, ha='center', transform=plt.gca().transAxes)
sns.lineplot(y_test.reset_index(drop=True), label='실제값', alpha=1)
sns.lineplot(test_pred, label='예측값', alpha=1)
plt.grid()
plt.show();

## 4-2.Catboost

In [ ]:
dt_full=pd.read_csv(f"{data_path}/dt_full_모델링.csv")

Logtarget=True # 타겟과 최근거래가를 로그화할지 말지

### 4-2-1. 타겟 로그화

In [ ]:
if(Logtarget==True):
    print("Target and recenttarget log transform")
    dt_full['target']=np.log1p(dt_full['target'])
    try: dt_full['최근거래가']=np.log1p(dt_full['최근거래가'])
    except:pass

def transform_log(pred,target=None):
    if(Logtarget==True):
        return np.expm1(pred),np.expm1(target)
    else:
        return pred,target

### 4-2-2. 년도 제거

In [ ]:
dt_full=dt_full[dt_full['계약년']>2019]
dt_full.reset_index(drop=True,inplace=True)

### 4-2-3. 최종 drop 컬럼 결정하기

In [ ]:
dt_full.columns

In [ ]:
drop_cols4=['시군구', '번지', '본번', '부번', '전용면적', '계약일', '도로명',
       '거래유형', 'k_관리방식', 'k_복도유형', 'k_난방방식', 'k_전체동수', 'k_전체세대수', 'k_건설사시공사',
       'k_시행사', 'k_연면적', 'k_주거전용면적', 'k_관리비부과면적', 'k_전용면적별세대현황60이하',
       'k_전용면적별세대현황60_85이하', 'k_85_135이하', 'k_135초과', '세대전기계약방법', '주차대수',
       '관리비_업로드', '분양세대수', '임대세대수',
       '일반관리-인원', '경비관리-인원', '청소관리-인원', '전기-수전용량', '승강기', '지상주차대수', '지하주차대수',
       'CCTV대수', '최고층수', '지하층수', '차량보유대수(전체)', '차량보유대수(전기차)', '계약년월일', 
       '풀아파트명', '주소',  '구순위', '동순위', '아파트순위', '3월평단가_std',
       '3월평단가_skew', '3월평단가_kurt', '신축여부', '세대당주차대수', '세대당승강기대수',
       '고층도', '동세대수', '월거래건수_mean', '월거래건수_std', '월거래건수_skew', '월거래건수_kurt',
       '아파트거래횟수5', '전용면적_mean', '전용면적_std', '전용면적_kurt', '전용면적_size',
       '전용면적범주_0', '전용면적범주_1', '전용면적범주_2', '전용면적범주_3', '전용면적범주_4', '범주4비율',
       '이전계약_diff', ]
dt_full.drop(drop_cols4,axis=1,inplace=True)

In [ ]:
dt_full.columns

### 4-2-4. Train,Test 분리

In [ ]:
dt_train = dt_full[dt_full['is_test']==0]
dt_test = dt_full[dt_full['is_test']==1]
dt_test['target'] = 0

dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)
assert dt_train.shape[1] == dt_test.shape[1] 

### 4-2-5. 범주형 변수 선택 

In [ ]:
num_cols,cat_cols,rest = [],[],[]

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]): num_cols.append(column) # 
    elif pd.api.types.is_object_dtype(dt_train[column]): cat_cols.append(column)
    else: rest.append(column)
    
print("연속형 변수:", num_cols)
print("범주형 변수:", cat_cols)
print("나머지 변수:", rest)

### 4-2-6. Catboost 모델 학습 및 검증

#### Optuna

In [ ]:
def objective(trial, X, y):

    cbrm_param = {
        'iterations':trial.suggest_int("iterations", 400, 1200),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',3, 7),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
    }
    model = CatBoostRegressor(**cbrm_param,random_state=624,task_type="GPU")

    folds = KFold(n_splits=5, random_state=624, shuffle=True)
    losses = []

    for train_idx, valid_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]

        X_valid = X.iloc[valid_idx, :]
        y_valid = y.iloc[valid_idx]

        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=25,cat_features=cat_cols)
        preds = model.predict(X_valid)
        loss = np.sqrt(mean_squared_error(y_valid, preds))
        losses.append(loss)
        
    return np.mean(losses)

def make_tune_model(train_data):
    opt_func = partial(objective, X=train_data[0], y=train_data[1]) 
    K = 5 
    sampler = TPESampler(seed=624)
    
    study = optuna.create_study(direction="minimize", sampler=sampler)

    study.optimize(opt_func, n_trials=30)
    
    print("Tuned train Score: %.4f" % study.best_value) # best score 출력
    print("Tuned params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들
    
    best_params = study.best_params
    best_model = CatBoostRegressor(**best_params,random_state=624,task_type="GPU")
    best_model.fit(train_data[0], train_data[1],cat_features=cat_cols)
    
    return best_model

#### 4-2-6-1. 전체 학습

In [ ]:
model_path="../model"

X_train=dt_train.drop(['target'], axis=1)
y_train=dt_train['target']
#model=make_tune_model([X_train,y_train])
#save_model('Total',model)
model=load_model('Total')
best_params = model.get_params()
model = CatBoostRegressor(**best_params)
model.fit(X_train, y_train,cat_features=cat_cols)
save_model('Total',model)

In [ ]:
model=load_model('Total')
importances = pd.Series(model.feature_importances_, index=list(X_train.columns))
importances = importances.sort_values(ascending=False)

plt.figure(figsize=(10,8))
plt.title("Feature Importances")
sns.barplot(x=importances, y=importances.index)
plt.show();

In [ ]:
y_pred=model.predict(X_train)
y_pred,y_train=transform_log(y_pred,y_train)
result=pd.DataFrame({'pred': y_pred, 'target': y_train})

plt.figure(figsize=(20, 6))
plt.plot(result.index, result['target'], label='Target')
plt.plot(result.index, result['pred'], label='Predictions', marker='x')
plt.title(f'Train Result by Total')
plt.xlabel('Index')
plt.ylabel('Values')
plt.legend()
plt.grid(True)
plt.show();

#### 4-2-6-2. 구별 학습 

In [ ]:
model_path="../model/구별"
gu_col=dt_train['구'].unique()
model_list=[]

for gu in gu_col:
    tmp_train=dt_train[dt_train['구']==gu]
    X_train=tmp_train.drop(['target'], axis=1)
    y_train=tmp_train['target']
    
    # model=make_tune_model([X_train,y_train])
    # save_model(f'{gu}',model)
    model=load_model(f'Catboost_{gu}')
    best_params = model.get_params()
    model = CatBoostRegressor(**best_params)
    model.fit(X_train, y_train,cat_features=cat_cols)
    save_model(f'Catboost_{gu}',model)
    
    model_list.append(model)

In [ ]:
for gu in gu_col:
    model=load_model(f"Catboost_{gu}")
    importances = pd.Series(model.feature_importances_, index=list(X_train.columns))
    importances = importances.sort_values(ascending=False)

    plt.figure(figsize=(10,8))
    plt.title(f"{gu} Feature Importances")
    sns.barplot(x=importances, y=importances.index)
    plt.show();

In [ ]:
answer=[]
for gu in gu_col:
    tmp_train=dt_train[dt_train['구']==gu]
    X_train=tmp_train.drop(['target'], axis=1)
    y_train=tmp_train['target']
    
    idx=X_train.index.copy()
    model=load_model(f'Catboost_{gu}')
    
    gu_pred = model.predict(X_train)
    
    gu_pred,y_train=transform_log(gu_pred,y_train)
    gu_pred=pd.DataFrame({'pred': gu_pred, 'target': y_train})
    gu_pred['idx']=idx
    answer.append(gu_pred)
    
gu_result = pd.concat(answer)
gu_result = gu_result.sort_values(by='idx')
gu_result.drop(['idx'],axis=1,inplace=True)
gu_result.reset_index(drop=True,inplace=True)

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(gu_result.index, gu_result['target'], label='Target')
plt.plot(gu_result.index, gu_result['pred'], label='Predictions', marker='x')
plt.title(f'Train Result by GU')
plt.xlabel('Index')
plt.ylabel('Values')
plt.legend()
plt.grid(True)
plt.show();

#### 4-2-6-3.면적별 학습

In [ ]:
model_path="../model/면적별"
size_col=dt_train['전용면적범주'].unique()
model_list=[]

for size in size_col:
    tmp_train=dt_train[dt_train['전용면적범주']==size]
    X_train=tmp_train.drop(['target'], axis=1)
    y_train=tmp_train['target']
    
    # model=make_tune_model([X_train,y_train])
    # save_model(f'size_{size}',model)
    
    model=load_model(f'Catboost{size}')
    best_params = model.get_params()
    model = CatBoostRegressor(**best_params)
    model.fit(X_train, y_train,cat_features=cat_cols)
    save_model(f'Catboost_{size}',model)
    
    model_list.append(model)

In [ ]:
for size in size_col:
    model=load_model(f'Catboost_{size}')
    importances = pd.Series(model.feature_importances_, index=list(X_train.columns))
    importances = importances.sort_values(ascending=False)

    plt.figure(figsize=(10,8))
    plt.title(f"{size} Feature Importances")
    sns.barplot(x=importances, y=importances.index)
    plt.show();

In [ ]:
model_list=[]

answer=[]
for size in size_col:
    tmp_train=dt_train[dt_train['전용면적범주']==size]
    X_train=tmp_train.drop(['target'], axis=1)
    y_train=tmp_train['target']
    
    idx=X_train.index.copy()
    model=load_model(f'Catboost_{size}')
    
    size_pred = model.predict(X_train)
    size_pred,y_train=transform_log(size_pred,y_train)
    
    size_pred=pd.DataFrame({'pred': size_pred, 'target': y_train})
    size_pred['idx']=idx
    answer.append(size_pred)
    
size_result = pd.concat(answer)
size_result = size_result.sort_values(by='idx')
size_result.drop(['idx'],axis=1,inplace=True)
size_result.reset_index(drop=True,inplace=True)

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(size_result.index, size_result['target'], label='Target')
plt.plot(size_result.index, size_result['pred'], label='Predictions', marker='x')
plt.title(f'Train Result by SIZE')
plt.xlabel('Index')
plt.ylabel('Values')
plt.legend()
plt.grid(True)
plt.show();

### 4-2-7. Catboost 모델 추론

#### 4-2-7-1. 전체 추론

In [ ]:
model_path="../model"
model=load_model("Total")
X_test = dt_test.drop(['target'], axis=1)
total_pred = model.predict(X_test)
total_pred,_=transform_log(total_pred,0)
total_result = pd.DataFrame(total_pred.astype(int), columns=["target"])
total_result.head(3)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, total_result['target']))

plt.figure(figsize=(20, 6))
plt.text(0.5, 0.9, f'RMSE: {rmse:.2f}', fontsize=20, ha='center', transform=plt.gca().transAxes)
plt.plot(total_result.index,y_test, label='Target')
plt.plot(total_result.index, total_result['target'], label='Pred')
plt.xlabel('Index')
plt.ylabel('Values')
plt.legend()
plt.ylim(0, 1800000)
plt.yticks([i for i in range(0, 1800001, 250000)])
plt.title("Test set Predict by TOTAL")
plt.grid(True)
plt.show();
#total_result.to_csv(f'{submission_path}/20240123_Cat_optuna_total.csv', index=False);

#### 4-2-7-2. 구별 추론

In [ ]:
model_path="../model/구별"
gu_col=dt_train['구'].unique()
model_list=[]

answer=[]
for gu in gu_col:
    tmp_test=dt_test[dt_test['구']==gu]
    X_test=tmp_test.drop(['target'], axis=1)
    
    idx=X_test.index.copy()
    model=load_model(f"Catboost_{gu}")

    gu_pred = model.predict(X_test)
    gu_pred,_=transform_log(gu_pred,0)
    
    gu_pred=pd.DataFrame({'target': gu_pred.astype(int)})
    gu_pred['idx']=idx
    answer.append(gu_pred)
    
gu_result = pd.concat(answer)
gu_result = gu_result.sort_values(by='idx')
gu_result.drop(['idx'],axis=1,inplace=True)
gu_result.reset_index(drop=True,inplace=True)
gu_result.head(3)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, gu_result['target']))

plt.figure(figsize=(20, 6))
plt.text(0.5, 0.9, f'RMSE: {rmse:.2f}', fontsize=20, ha='center', transform=plt.gca().transAxes)
plt.plot(gu_result.index,y_test, label='Target')
plt.plot(gu_result.index, gu_result['target'], label='Pred')
plt.xlabel('Index')
plt.ylabel('Values')
plt.legend()
plt.ylim(0, 1800000)
plt.yticks([i for i in range(0, 1800001, 250000)])
plt.title("Test set Predict by Gu")
plt.grid(True)
plt.show();
#gu_result.to_csv(f'{submission_path}/20240123_Cat_optuna_gu.csv', index=False);

#### 4-2-7-3. 면적별 추론

In [ ]:
model_path="../model/면적별"
size_col=dt_train['전용면적범주'].unique()
model_list=[]

answer=[]
for size in size_col:
    tmp_test=dt_test[dt_test['전용면적범주']==size]
    X_test=tmp_test.drop(['target'], axis=1)

    idx=X_test.index.copy()
    model=load_model(f"Catboost_{size}")
    
    size_pred = model.predict(X_test)
    size_pred,_=transform_log(size_pred,0)

    size_pred=pd.DataFrame({'target': size_pred.astype(int)})
    size_pred['idx']=idx
    answer.append(size_pred)
    
size_result = pd.concat(answer)
size_result = size_result.sort_values(by='idx')
size_result.drop(['idx'],axis=1,inplace=True)
size_result.reset_index(drop=True,inplace=True)
size_result.head(3)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, size_result['target']))

plt.figure(figsize=(20, 6))
plt.text(0.5, 0.9, f'RMSE: {rmse:.2f}', fontsize=20, ha='center', transform=plt.gca().transAxes)
plt.plot(size_result.index,y_test, label='Target')
plt.plot(size_result.index, size_result['target'], label='Pred')
plt.xlabel('Index')
plt.ylabel('Values')
plt.legend()
plt.ylim(0, 1800000)
plt.yticks([i for i in range(0, 1800001, 250000)])
plt.title("Test set Predict by SIZE")
plt.grid(True)
plt.show();
#size_result.to_csv(f'{submission_path}/20240123_Cat_optuna_size.csv', index=False);

# 6. Experiment

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(adjust_result.index, adjust_result['target'], label='Target')
plt.xlabel('Index')
plt.ylabel('Values')
plt.ylim(0, 1800000)
plt.yticks([i for i in range(0, 1800001, 250000)])
plt.legend()
plt.title("Test set Predict*Weight")
plt.grid(True)
plt.show();
#adjust_result.to_csv(f'{submission_path}/20240121_cat_by gu_recent+adjust.csv', index=False);

#### 가중치 조절 size별 모델

In [ ]:
dt_test_tmp=dt_test.copy()

model_path="../model"
dt_train.columns
model=load_model("Total")
X_test = dt_test_tmp.drop(['target'], axis=1)
size_tmp=X_test['전용면적범주'].values
total_pred = model.predict(X_test)
total_pred  = np.expm1(total_pred )
total_result = pd.DataFrame(total_pred.astype(int), columns=["target"])
#total_result['전용면적범주']=size_tmp
#total_result['real_target']=test_target
total_result.head(3)
#total_result=total_result.sort_values(by='전용면적범주',ignore_index=True)


total_result.loc[total_result['target'] > 0, 'target'] *= 1.1
#rmse = np.sqrt(mean_squared_error(total_result['real_target'], total_result['target']))

plt.figure(figsize=(20, 6))
#plt.text(0.5, 0.9, f'RMSE: {rmse:.2f}', fontsize=20, ha='center', transform=plt.gca().transAxes)
#plt.plot(total_result.index,total_result['real_target'], label='Target')
plt.plot(total_result.index, total_result['target'], label='Pred')
plt.xlabel('Index')
plt.ylabel('Values')
plt.legend()
plt.ylim(0, 1800000)
plt.yticks([i for i in range(0, 1800001, 250000)])
plt.title("Test set Predict by total")
plt.grid(True)
plt.show();
#size_result.to_csv(f'{submission_path}/20240123_Cat_optuna_size.csv', index=False);

In [ ]:
total_result['target']=total_result['target'].astype('int')
total_result.to_csv(f'{submission_path}/20240123_Cat_optuna_total_1_1.csv', index=False);

In [ ]:
our=pd.read_csv("../submission/20240123_Cat_optuna_total_1_1.csv")
hyo=pd.read_csv("sub.csv")
base=pd.read_csv("baseline.csv.csv")
yc=pd.read_csv("youngcheon.csv")

In [ ]:
plt.figure(figsize=(20, 6))
#plt.plot(our.index, our['target'], label='our')
#plt.plot(hyo.index, hyo['target'], label='hyo')
plt.plot(yc.index, yc['target'], label='yc')
plt.plot(our.index, our['target'], label='our')
plt.title('Real Target')
plt.xlabel('Index')
plt.ylabel('Values')
plt.ylim(0, 1800000)
plt.yticks([i for i in range(0, 1800001, 250000)])
plt.legend()
plt.grid(True)
plt.show();

In [ ]:
RMSE = mean_squared_error(our['target'], hyo['target'])**0.5
RMSE